In [ ]:
!pip install apache-beam[gcp]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.8/193.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.7/184.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d nagasai524/nyc-taxi-trip-records-from-jan-2023-to-jun-2023
!unzip nyc-taxi-trip-records-from-jan-2023-to-jun-2023.zip -d data/

Dataset URL: https://www.kaggle.com/datasets/nagasai524/nyc-taxi-trip-records-from-jan-2023-to-jun-2023
License(s): U.S. Government Works
 99% 404M/407M [00:06<00:00, 56.2MB/s]
100% 407M/407M [00:06<00:00, 67.9MB/s]
Archive:  nyc-taxi-trip-records-from-jan-2023-to-jun-2023.zip
  inflating: data/nyc_yellow_taxi_trip_records_from_Jan_to_Aug_2023.csv  


In [ ]:
import pandas as pd

df_sample = pd.read_csv('/content/data/nyc_yellow_taxi_trip_records_from_Jan_to_Aug_2023.csv', usecols=['tpep_pickup_datetime', 'trip_distance', 'fare_amount'], nrows=1000)

df_sample.to_csv('nyc_taxi_sample.csv', index=False)


In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

def format_taxi_data(element):
    """Splits CSV line into fields and returns a dictionary."""
    fields = element.split(',')
    return {
        'pickup_datetime': fields[0],
        'trip_distance': float(fields[1]),
        'fare_amount': float(fields[2])
    }

class FilterShortTrips(beam.DoFn):
    """Filter trips shorter than 1 mile."""
    def process(self, element):
        if element['trip_distance'] >= 1.0:
            yield element

pipeline_options = PipelineOptions()

with beam.Pipeline(options=pipeline_options) as p:

    taxi_data = (p
                 | 'Read CSV File' >> beam.io.ReadFromText('nyc_taxi_sample.csv', skip_header_lines=1)
                 | 'Format Taxi Data' >> beam.Map(format_taxi_data)  # Composite Transform
                )

    filtered_trips = (taxi_data
                      | 'Filter Short Trips' >> beam.ParDo(FilterShortTrips())
                     )

    windowed_trips = (filtered_trips
                      | 'Window' >> beam.WindowInto(beam.window.FixedWindows(60))  # 60-second windows for demonstration
                     )

    output = (windowed_trips
              | 'Format Output' >> beam.Map(lambda x: f"Distance: {x['trip_distance']}, Fare: {x['fare_amount']}")
              | 'Write Output' >> beam.io.WriteToText('output/filtered_trips.txt')
             )

p.run().wait_until_finish()


'DONE'

In [ ]:
import apache_beam.transforms.window as window

with beam.Pipeline(options=pipeline_options) as p:

    taxi_data = (p
                 | 'Read CSV File' >> beam.io.ReadFromText('nyc_taxi_sample.csv', skip_header_lines=1)
                 | 'Format Taxi Data' >> beam.Map(format_taxi_data)
                )

    windowed_trips = (taxi_data
                      | 'Window' >> beam.WindowInto(beam.window.FixedWindows(60),  # 60-second windows
                                                    trigger=beam.trigger.AfterProcessingTime(10),
                                                    accumulation_mode=beam.trigger.AccumulationMode.DISCARDING)
                     )

    output = (windowed_trips
              | 'Format Output' >> beam.Map(lambda x: f"Distance: {x['trip_distance']}, Fare: {x['fare_amount']}")
              | 'Write Output' >> beam.io.WriteToText('output/triggered_trips.txt')
             )

p.run().wait_until_finish()


'DONE'